In [1]:
%matplotlib inline
import numpy as np
import gp_grief.kern
import matplotlib.pyplot as plt
import GPy.kern

In [2]:
def rastrigin(x,lin_term=None):
    """
    Rastrigin test function
    input should be in range [-5.12, 5.12]
    if x in range [0,1], can transform by rastrigin((x*2-1)*5.12)
    if lin_term is not None then will add a linear term to the first dimension. This helps
    to make the function non-symetric wrt the input dimensions
    """
    assert x.ndim == 2
    d = x.shape[1]
    f = 10*d
    for i in range(d):
        f = f+(np.power(x[:,i,None],2) - 10*np.cos(2*np.pi*x[:,i,None]));
    if lin_term is not None:
        f += lin_term*x[:,(0,)]
    return f

# Standard Kernels
Compare the covariance Matricies with GPy Kernels

In [3]:
for d in [1,10]:
    np.random.seed(0)
    N = 100
    x = np.random.uniform(size=(N,d)) # generate dataset
    y = rastrigin((x*2-1)*5.12)

    # RBF
    np.testing.assert_array_almost_equal(
        gp_grief.kern.RBF(d, lengthscale=0.5, variance=0.5).cov(x),
        GPy.kern.RBF(    d, lengthscale=0.5, variance=0.5).K(x))
    
    # Exponential
    np.testing.assert_array_almost_equal(
        gp_grief.kern.Exponential(d, lengthscale=0.5, variance=0.5).cov(x),
        GPy.kern.Exponential(    d, lengthscale=0.5, variance=0.5).K(x))
    
    # Matern32
    np.testing.assert_array_almost_equal(
        gp_grief.kern.Matern32(d, lengthscale=0.5, variance=0.5).cov(x),
        GPy.kern.Matern32(    d, lengthscale=0.5, variance=0.5).K(x))
    
    # Matern52
    np.testing.assert_array_almost_equal(
        gp_grief.kern.Matern52(d, lengthscale=0.5, variance=0.5).cov(x),
        GPy.kern.Matern52(    d, lengthscale=0.5, variance=0.5).K(x))
    
    # GPy - RBF
    np.testing.assert_array_almost_equal(
        gp_grief.kern.GPyKernel(d, kernel="RBF", lengthscale=0.5, variance=0.5).cov(x),
        GPy.kern.RBF(          d, lengthscale=0.5, variance=0.5).K(x))
    
    if d == 1: # these are only for 1d
        # GPy - PeriodicMatern32
        np.testing.assert_array_almost_equal(
            gp_grief.kern.GPyKernel(d, kernel="PeriodicMatern32", lengthscale=0.5, variance=0.5).cov(x),
            GPy.kern.PeriodicMatern32(          d, lengthscale=0.5, variance=0.5).K(x))

# Combining Kernels
Compare the covariance Matricies with GPy Kernels. Let the kernel be
$$ k = (k_0 k_1 + k_2)k_3$$

## Test with in-house kernels

In [4]:
# generate data
np.random.seed(0)
d = 5
N = 100
x = np.random.uniform(size=(N,d)) # generate dataset
y = rastrigin((x*2-1)*5.12)

# consider cases where the base kernels are the same and different
for case in ['constant_bases', 'different_bases']:
    print ''
    print case
    print '*' * 80
    
    # first define the base kernels
    if case == 'constant_bases':
        kb_gpy = [GPy.kern.RBF(d, lengthscale=0.5, variance=0.5) for i in range(4)]
        # for gp_grief I want to define it once and test copying it
        tmp = gp_grief.kern.RBF(d, lengthscale=0.5, variance=0.5) 
        kb_kml = [tmp.copy() for i in range(4)]
    elif case == 'different_bases':
        kb_gpy = [GPy.kern.RBF(    d, lengthscale=0.5*i+0.5, variance=0.5*i+0.5, name='k%d' % i) 
                  for i in range(4)]
        kb_kml = [gp_grief.kern.RBF(d, lengthscale=0.5*i+0.5, variance=0.5*i+0.5, name='k%d' % i) 
                  for i in range(4)]
    else:
        assert False

    # then combine the base kernels
    k_gpy = ((kb_gpy[0] * kb_gpy[1]) + kb_gpy[2]) * kb_gpy[3]
    k_kml = ((kb_kml[0] * kb_kml[1]) + kb_kml[2]) * kb_kml[3]
    
    # print the kernels
    print 'gpy:'
    print k_gpy
    print 'gp_grief'
    print k_kml

    # check to ensure they give the same kernel covariance matrix
    np.testing.assert_array_almost_equal( k_kml.cov(x), k_gpy.K(x) )
    
    # test training
    m = gp_grief.models.GPRegression(x,y,k_kml)
    m.optimize(max_iters=5) # just go for a few iterations to make sure working with getter/setter of params
    m.checkgrad()


constant_bases
********************************************************************************
gpy:
  mul.                       |  value  |  constraints  |  priors
  sum.mul.rbf.variance       |    0.5  |      +ve      |        
  sum.mul.rbf.lengthscale    |    0.5  |      +ve      |        
  sum.mul.rbf_1.variance     |    0.5  |      +ve      |        
  sum.mul.rbf_1.lengthscale  |    0.5  |      +ve      |        
  sum.rbf.variance           |    0.5  |      +ve      |        
  sum.rbf.lengthscale        |    0.5  |      +ve      |        
  rbf.variance               |    0.5  |      +ve      |        
  rbf.lengthscale            |    0.5  |      +ve      |        
gp_grief

RBF kernel
| Name        |   Value | Constraint   |
|-------------+---------+--------------|
| variance    |     0.5 | +ve          |
| lengthscale |     0.5 | +ve          |

mul with child:

RBF kernel
| Name        |   Value | Constraint   |
|-------------+---------+--------------|
| variance    |  

## Test using Wrapped `GPy` Kernels

In [6]:
# generate data
np.random.seed(0)
d = 5
N = 100
x = np.random.uniform(size=(N,d)) # generate dataset
y = rastrigin((x*2-1)*5.12)

# get base kernels
kb_gpy = [GPy.kern.RBF(    d, lengthscale=0.5*i+0.5, variance=0.5*i+0.5, name='k%d' % i) 
                  for i in range(4)]
# then combine the base kernels
k_gpy = ((kb_gpy[0] * kb_gpy[1]) + kb_gpy[2]) * kb_gpy[3]

# Wrap the GPy kernel
k_kml = gp_grief.kern.GPyKernel(d, kernel=k_gpy)

# print the kernels
print 'gpy:'
print k_gpy
print 'gp_grief:'
print k_kml

# check to ensure they give the same kernel covariance matrix
np.testing.assert_array_almost_equal( k_kml.cov(x), k_gpy.K(x) )

# test training
m = gp_grief.models.GPRegression(x,y,k_kml)
m.checkgrad()
m.optimize(max_iters=5); # just go for a few iterations to make sure working with getter/setter of params

gpy:
  mul.                    |  value  |  constraints  |  priors
  sum.mul.k0.variance     |    0.5  |      +ve      |        
  sum.mul.k0.lengthscale  |    0.5  |      +ve      |        
  sum.mul.k1.variance     |    1.0  |      +ve      |        
  sum.mul.k1.lengthscale  |    1.0  |      +ve      |        
  sum.k2.variance         |    1.5  |      +ve      |        
  sum.k2.lengthscale      |    1.5  |      +ve      |        
  k3.variance             |    2.0  |      +ve      |        
  k3.lengthscale          |    2.0  |      +ve      |        
gp_grief:

GPy - <GPy.kern.src.prod.Prod object at 0x7f50aff05110> kernel
| Name        |   Value | Constraint   |
|-------------+---------+--------------|
| variance    |     0.5 | ['+ve']      |
| lengthscale |     0.5 | ['+ve']      |
| variance    |     1   | ['+ve']      |
| lengthscale |     1   | ['+ve']      |
| variance    |     1.5 | ['+ve']      |
| lengthscale |     1.5 | ['+ve']      |
| variance    |     2   | ['+ve']  